In [10]:
import os
from pathlib import Path
import wandb
import sys
sys.path.append("/home/jc053/GIT/mri-post-op-segmentation/")
from utils.helper_functions import run_command, evaluate_fold, log_metrics_to_wandb, prepare_nnunet_data
from src.trainer_nnUnet import nnUNetTrainerV2_WandB
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.device_count())

2.4.0+cu121
12.1
90100
True
2


## Set config & environment variables

In [11]:
# Configuration
TASK_NAME = "BraTS2024_PostSurgery"
TASK_ID = 901  # Assign a unique task ID
BASE_DIR = Path("/mnt/93E8-0534/JuanCarlos/BraTS-GLI-2024/organized_dataset")  # Adjust this path
NUM_GPUs = 2
WANDB_PROJECT = "BraTS2024_nnUNet"
MODEL = "nnUNet_test"

# Set environment variables
NNUNET_RAW = BASE_DIR / MODEL / "nnUNet_raw_data_base" / "nnUNet_raw_data" / f"Task{TASK_ID}_{TASK_NAME}"
os.environ["nnUNet_raw_data_base"] = str(BASE_DIR / MODEL / "nnUNet_raw_data_base")
os.environ["nnUNet_preprocessed"] = str(BASE_DIR / MODEL / "nnUNet_preprocessed")
os.environ["RESULTS_FOLDER"] = str(BASE_DIR / MODEL / "nnUNet_trained_models")
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# Step 0: Data Preparation (if data is not prepared already)

In [ ]:
prepare_nnunet_data(NNUNET_RAW, BASE_DIR, TASK_NAME, max_cases=10)

## Step 1: Data Planning and Preprocessing (if not done already, to be in the nnUnet configuration style)

In [ ]:
success = run_command(f"nnUNet_plan_and_preprocess -t {TASK_ID} --verify_dataset_integrity")
if not success:
   print("Dataset planning and preprocessing failed.")
   exit(1)

## Step 2: Model Training (3D fullres)

In [13]:
# Initialize wandb
wandb.init(project=WANDB_PROJECT, name=f"{TASK_NAME}_run")

# # Log configuration
wandb.config.update({
    "task_name": TASK_NAME,
    "task_id": TASK_ID,
    "num_gpus": NUM_GPUs
})

success = run_command(f"CUDA_VISIBLE_DEVICES=0,1 nnUNet_train 3d_fullres nnUNetTrainerV2 {TASK_ID} 0")
if not success:
    wandb.finish()
    exit(1)

# Evaluate and log metrics
TRAINED_MODELS_DIR = BASE_DIR / MODEL / "nnUNet_trained_models" / "nnUNet" / "3d_fullres" / f"Task{TASK_ID}_{TASK_NAME}"
output_fold = TRAINED_MODELS_DIR / "fold_0"
dsc, hd95 = evaluate_fold(str(output_fold), str(BASE_DIR))
log_metrics_to_wandb({"3d_fullres_DSC": dsc, "3d_fullres_HD95": hd95}, step=0)

# Log final model files
model_artifact = wandb.Artifact(f"{TASK_NAME}_model", type="model")
model_artifact.add_dir(str(TRAINED_MODELS_DIR / "fold_0"))
wandb.log_artifact(model_artifact)

print("Training complete!")
wandb.finish()




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: 'T1', 1: 'T1ce', 2: 'T2', 3: 'FLAIR'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT'), (1, 'nonCT'), (2, 'nonCT'), (3, 'nonCT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [5, 5, 5]

## Step 3: Inference

In [9]:
INPUT_FOLDER = str(BASE_DIR / MODEL / "nnUNet_raw_data_base" /"nnUNet_raw_data"/f"Task{TASK_ID}_{TASK_NAME}"/"imagesTs")
OUTPUT_FOLDER = str(BASE_DIR / MODEL / "inference_results")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

inference_command = (
    f"nnUNet_predict -i {INPUT_FOLDER} -o {OUTPUT_FOLDER} "
    f"-t {TASK_ID} -m 3d_fullres -f 0 --save_npz"
)
success = run_command(inference_command)
if not success:
    print("Inference failed.")
    exit(1)

print(f"Inference complete. Results saved in {OUTPUT_FOLDER}")



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /mnt/93E8-0534/JuanCarlos/BraTS-GLI-2024/organized_dataset/nnUNet_test/nnUNet_trained_models/nnUNet/3d_fullres/Task901_BraTS2024_PostSurgery/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 4 input modalities for each image
Found 10 unique case ids, here are some examples: ['02834_100' '02773_101' '00553_100' '02135_101' '02089_101' '00553_101'
'02089_101' '02834_101' '00553_100' '00553_101']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 10
number of cases that still need to be predicted: 10
emptying cuda cache
loading parameters f